In [1]:
import numpy as np
import pandas as pd

list = []
# sparsowanie loga to listy słowników wypłaszczająć termal - 1 linia 1 słownik
with open('data_provider1.log', 'r') as file_in:
    for line in file_in:
        s1 = line.split(" INFO data_provider - handleData: ")
        if len(s1) > 1 and len(s1[1]) > 30:
            e = eval(s1[1])
            e['date'] = s1[0]
            d = e.pop('thermal')
            for key in d:
                e[key] = d[key]

            list.append(e)

f = open("output.txt", "w")
f.write(str(list))
f.close()

with open('output.txt') as f:
    data = eval(f.read())
# wczytanie w pandasowy dataframe
df = pd.DataFrame(data)
pd.set_option('display.max_columns', None)
# dodanie kolumny NEW_ID dla lepszej logistyki odnajdywania logów
df.insert(0, 'New_ID', range(1, 1 + len(df)))

# check dla minimalnych temperatur poniżej zdefiniowaniej niżej wartości
min_check = df.query("min_temp < 16.4")
z = min_check["New_ID"].unique()
print("test minimalnej temp wskazał wpisy id:")
print(z)
print("")

# check dla maksymalnych temperatur powyżej zdefiniowaniej niżej wartości:
max_check = df.query("max_temp > 34")
y = max_check["New_ID"].unique()
print("test max temp wskazał wpisy id:")
print(y)
print("")

pd.options.mode.chained_assignment = None
# rozpakowanie temp_array do rzędów z 1 listą
df3 = df.explode('temp_array')
# rozpakowanie listy do wartości z temperaturą
df4 = df3.explode("temp_array")
# obliczenie różnicy pomiędzy rzędami
df4['temp_diff'] = df4['temp_array'].diff()

# check dla różnicy wwartości temp w temp_array powyżej zdefiniowaniej niżej wartości:
df5 = df4.query("temp_diff > 5")
x = df5["New_ID"].unique()
print("test różnic temp wskazał wpisy id:")
print(x)
print("")

# kopiowanie wyników odchylonych ID z trzech testów i zapisanie do listy
new_arr = np.append(x, y)
wynik = np.append(new_arr, z)
wynik = wynik.tolist()
print("lista odchylonych id z 3 testów:")
print(wynik)
print("")

# filtrowanie pierwotnego dataframa do wyników odchylonych temp w logach po id:
df_filtered = df[df['New_ID'].isin(wynik)]
df_filtered.to_csv("out.txt", index=False)
print("odchylone logi zapisane zostały jako out.txt")


test minimalnej temp wskazał wpisy id:
[  64   70  169  280 1075 1237]

test max temp wskazał wpisy id:
[ 744 1788 2652 3360 3501]

test różnic temp wskazał wpisy id:
[1048 1237 1696]

lista odchylonych id z 3 testów:
[1048, 1237, 1696, 744, 1788, 2652, 3360, 3501, 64, 70, 169, 280, 1075, 1237]

odchylone logi zapisane zostały jako out.txt


In [2]:
df_filtered

,New_ID,target_id,mode,timestamp,date,ok,width,height,temp_array,max_temp,min_temp,mean_temp,median_temp,detail,warning,ambient_temp,pressure,humidity
63,64,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652105885,2022-05-09 07:18:05.297,True,24,32,"[[21.53, 21.98, 22.19, 22.52, 21.99, 21.95, 21...",25.77,16.35,22.15,22.35,Normal ops,False,24.078019,922.619817,14.559800
69,70,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652106005,2022-05-09 07:20:05.527,True,24,32,"[[21.74, 21.19, 22.01, 22.17, 21.81, 21.8, 22....",25.30,16.18,21.99,22.19,Normal ops,False,23.952647,922.548696,14.679863
168,169,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652107988,2022-05-09 07:53:08.249,True,24,32,"[[22.06, 22.21, 22.31, 22.28, 22.33, 22.16, 22...",25.67,16.31,22.19,22.50,Normal ops,False,24.288643,922.677303,14.369310
279,280,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652110211,2022-05-09 08:30:11.462,True,24,32,"[[21.82, 22.05, 22.06, 22.24, 22.31, 21.81, 21...",25.34,16.34,22.14,22.36,Normal ops,False,24.037900,922.996717,14.639226
743,744,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652119464,2022-05-09 11:04:24.542,True,24,32,"[[26.1, 26.05, 26.12, 26.72, 26.56, 26.35, 25....",34.09,24.87,25.83,25.57,Normal ops,False,26.742172,923.202534,13.558102
1047,1048,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652125592,2022-05-09 12:46:32.931,True,24,32,"[[24.43, 23.53, 23.42, 23.41, 23.38, 22.9, 23....",29.41,17.98,24.08,23.85,Normal ops,False,25.146191,922.057160,15.600029
1074,1075,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652126133,2022-05-09 12:55:33.743,True,24,32,"[[22.93, 22.16, 22.76, 22.27, 22.61, 22.06, 22...",26.68,16.36,22.80,22.89,Normal ops,False,24.434075,921.907719,15.967221
1236,1237,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652129378,2022-05-09 13:49:38.270,True,24,32,"[[24.85, 23.94, 23.78, 24.37, 24.27, 24.17, 23...",31.97,7.42,24.63,24.44,Normal ops,False,26.244467,921.667486,14.899177
1695,1696,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652138571,2022-05-09 16:22:51.177,True,24,32,"[[23.17, 23.45, 22.95, 23.14, 23.12, 22.83, 23...",26.96,17.24,23.51,23.45,Normal ops,False,25.547386,920.726234,15.654000
1787,1788,66ce3d64-7dbb-402e-9620-6f8c0525c5ab,calibrate,1652140373,2022-05-09 16:52:53.825,True,24,32,"[[26.44, 26.55, 26.44, 27.18, 27.0, 26.64, 26....",34.25,24.48,26.02,25.78,Normal ops,False,26.737131,920.977154,14.481089
